In [7]:
import pandas as pd

In [8]:
path_ages = "data/db/potato_dm_v404_all_pm_un_tephra_ltr-tir_age_summary.tsv"
path_tes = "data/db/out.gff"

In [9]:
df_ages = pd.read_csv(path_ages, sep='\t')
print(len(df_ages.index))
df_ages.head(2)

1614


,Superfamily,Family,Family_size,ElementID,Divergence,Age,Ts:Tv
0,Copia,RLC_family0,43,RLC_family0_exemplar,0.0471,2355000,3.2242
1,Copia,RLC_family1,33,RLC_family1_exemplar,0.0212,1060000,4.1588


In [51]:
df_tes = pd.read_csv(path_tes, sep='\t', comment='#', header=None)
df_tes.columns = ['seqname' , 'source' , 'feature' , 'start' , 'end' , 'score' , 'strand' , 'frame' , 'attribute']
print(len(df_tes.index))
df_tes.head(2)

955859


,seqname,source,feature,start,end,score,strand,frame,attribute
0,ChrUn,BLASTN,similarity,1,725,0,+,.,ID=DTX_singleton_family2245_terminal_inverted_...
1,ChrUn,BLASTN,similarity,823,1144,0,+,.,ID=DTX_singleton_family3808_terminal_inverted_...


In [ ]:
dfs = []
for k,v in df_ages.iterrows():
    te_type, family = v.Family.split('_')
    family = '_' + family + '_'
    res = df_tes[(df_tes.attribute.str.contains(te_type)) & (df_tes.attribute.str.contains(family))]
    dfs.append(res)
    print(k)

In [ ]:
df_res = pd.concat(dfs)
print(len(df_res.index))
df_res.head(5)

In [ ]:
df_res = df_res[['seqname','start','end','attribute']]
df_res = df_res[(df_res.seqname != 'chr00') & (df_res.seqname != 'ChrUn')]
print(len(df_res.index))
df_res.head(5)

In [ ]:
df_res.to_csv('data/ltrs.csv', index=None, sep='\t')

In [ ]:
df_res['family_number'] = df_res['attribute'].str.split('_family').str[1]
df_res['family'] = df_res['attribute'].str.split('_family').str[0]

In [ ]:
df_res.head(4)

In [ ]:
df_res['family_full'] = df_res['family'].str.split('=').str[-1] + "_family" + df_res['family_number'].str.split('_').str[0]

In [57]:
print(len(df_res.index))
df_res = df_res[['seqname','start','end','family_full']]
df_res.head(4)

182884


,seqname,start,end,family_full
195822,chr01,1601238,1601347,RLC_family0
196503,chr01,2283977,2284134,RLC_family0
196504,chr01,2285047,2285997,RLC_family0
196815,chr01,2707450,2707537,RLC_family0


In [58]:
df_res.to_csv('data/ltrs.csv', index=None, sep='\t')

In [4]:
df_res = pd.read_csv('data/ltrs.csv', sep='\t')
df_res_2 = df_res

In [5]:
#df_res_2 = df_res[df_res.family_full.str.contains('RLC')]
print(len(df_res_2.index))

182884


In [10]:
df_ages_merge = pd.merge(df_res_2,df_ages[['Family','Age']],left_on='family_full',right_on='Family',how='left')

In [11]:
df_ages_merge.head(2)

,seqname,start,end,family_full,Family,Age
0,chr01,1601238,1601347,RLC_family0,RLC_family0,2355000.0
1,chr01,2283977,2284134,RLC_family0,RLC_family0,2355000.0


In [12]:
df_ages_merge = df_ages_merge[['seqname','start','end','Age']]

In [13]:
df_ages_merge = df_ages_merge.rename(columns={'seqname':'Chr','start':'Start','end':'End','Age':'Value'})

In [14]:
df_ages_merge.to_csv('data/ltrs_ages.csv', index=None, sep='\t')